In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [65]:
path = "F:\Pfactor_clean"
file_name_list = os.listdir(path)
for year in ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']:
    factor_concat = {}
    pbar = tqdm(file_name_list)
    for file_name in pbar:
        pbar.set_description(f"{year}")
        factor_name = file_name[:-8]
        factor = pd.read_feather(f"{path}\\{file_name}").set_index('dt')
        factor = factor.loc[f"{year}-01-01":f"{year}-12-31", :]
        factor_concat[factor_name] = factor
    factor_concat = pd.concat(factor_concat, axis=1)
    factor_concat.to_pickle(f"./data/factor_concat_{year}.pkl")
del factor_concat

2023: 100%|██████████| 982/982 [04:25<00:00,  3.70it/s]


In [66]:
quantile = 2
stock_return = pd.read_pickle("./data/stock_return.pkl")
for year in ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']:
    print(year)
    factor_stack = pd.read_pickle(f"./data/factor_concat_{year}.pkl").stack()
    quantile_return = stock_return.stack()
    quantile_return = quantile_return.groupby('dt').apply(
        lambda x: pd.qcut(x, np.arange(quantile + 1) / quantile, np.arange(quantile))
    )
    common_idx = np.intersect1d(factor_stack.index, quantile_return.index)
    factor_stack = factor_stack.loc[common_idx, :]
    quantile_return = quantile_return[common_idx]
    factor_stack.to_pickle(f"./data/processed_data/factor_stack_{year}.pkl")
    quantile_return.to_pickle(f"./data/processed_data/quantile_return_{year}.pkl")

2015
2016
2017
2018
2019
2020
2021
2022
2023


In [153]:
# factor_stack = pd.read_pickle(f"./data/processed_data/factor_stack_2023.pkl")
code_num_dict = pd.read_pickle(f"./data/processed_data/code_num_dict.pkl")
factor_stack['code'] = factor_stack.reset_index()['code'].str.split('.', expand=True)[1].values
factor_stack.iloc[:,982]

dt          code     
2023-01-03  000001.SZ    SZ
            000002.SZ    SZ
            000006.SZ    SZ
            000007.SZ    SZ
            000008.SZ    SZ
                         ..
2023-03-29  688798.SH    SH
            688799.SH    SH
            688800.SH    SH
            688819.SH    SH
            688981.SH    SH
Name: code, Length: 252265, dtype: object

In [48]:
stock_return = pd.read_pickle("./data/stock_return.pkl")
stock_return.tail()

,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,688786.SH,688787.SH,688788.SH,688789.SH,688793.SH,688798.SH,688799.SH,688800.SH,688819.SH,688981.SH
dt,,,,,,,,,,,,,,,,,,,,,
2023-03-27,-0.001581,-0.001949,0.011051,-0.010791,-0.039486,-0.021656,-0.008148,-0.010336,-0.041426,-0.017294,...,-0.038314,0.114078,-0.004125,-0.067471,-0.053966,-0.006448,0.014540,-0.023818,0.012867,0.090452
2023-03-28,-0.010211,-0.017520,-0.022167,-0.011453,-0.040195,-0.011728,0.008215,0.000889,-0.045917,-0.017806,...,-0.038921,0.007145,0.012863,-0.022989,-0.041505,0.117735,-0.003170,-0.036197,-0.003668,0.168937
2023-03-29,0.001595,-0.007793,-0.019236,-0.005518,-0.009866,-0.023299,0.008510,0.010705,-0.077223,-0.008732,...,-0.056249,-0.168016,0.142500,-0.035468,0.008901,0.137796,-0.001567,-0.003536,-0.013972,0.220522
2023-03-30,-0.003942,0.009618,0.025745,0.005549,0.019821,-0.011835,0.025553,0.003452,-0.039399,0.019888,...,-0.042899,-0.009663,0.130799,-0.045632,-0.014048,0.138248,0.032687,0.041307,-0.023289,0.228106
2023-03-31,0.011952,0.023620,-0.016041,-0.010976,-0.009807,-0.023513,-0.004331,0.000888,-0.089168,0.012930,...,-0.022401,-0.154377,0.086582,-0.045515,-0.047043,0.123679,0.049176,0.053028,-0.021342,0.192132


In [5]:
quantile = 2
quantile_return = stock_return.stack()
quantile_return = quantile_return.groupby('dt').apply(
    lambda x: pd.qcut(x, np.arange(quantile + 1) / quantile, np.arange(quantile))
)
quantile_return.head()

dt          code     
2014-01-02  000001.SZ    1
            000002.SZ    0
            000004.SZ    1
            000005.SZ    1
            000006.SZ    0
dtype: category
Categories (2, int64): [0 < 1]

In [6]:
common_idx = np.intersect1d(factor_stack.index, quantile_return.index)
factor_stack = factor_stack.loc[common_idx, :]
quantile_return = quantile_return[common_idx]

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier

max_epochs = 30
for year in [2018, 2019, 2020, 2021, 2022]:
    print(year)
    x_train = pd.read_pickle(f"./data/processed_data/factor_stack_{year - 3}.pkl")
    x_train = pd.concat([x_train, pd.read_pickle(f"./data/processed_data/factor_stack_{year - 2}.pkl")])
    x_train = pd.concat([x_train, pd.read_pickle(f"./data/processed_data/factor_stack_{year - 1}.pkl")])
    y_train = pd.read_pickle(f"./data/processed_data/quantile_return_{year - 3}.pkl")
    y_train = pd.concat([y_train, pd.read_pickle(f"./data/processed_data/quantile_return_{year - 2}.pkl")])
    y_train = pd.concat([y_train, pd.read_pickle(f"./data/processed_data/quantile_return_{year - 1}.pkl")])
    x_train, x_valid, y_train, y_valid = train_test_split(x_train.values, y_train.values)
    model = TabNetClassifier(
        device_name='cuda'
    )
    model.fit(x_train, y_train,
              eval_set=[(x_valid, y_valid)],
              drop_last=False,
              max_epochs=max_epochs)
    model.save_model(f"./log/tabnet/{year}")
    del x_train, x_valid, y_train, y_valid

    x_test = pd.read_pickle(f"./data/processed_data/factor_stack_{year}.pkl")
    score = model.predict_proba(x_test.values)
    score = pd.DataFrame(score[:, 1], index=pd.MultiIndex.from_tuples(x_test.index))
    score.index.names = ['dt', 'code']
    score = score.unstack()
    score.columns = score.columns.droplevel(0)
    score.to_pickle(f"F:\Multifactor_Project\score_{year}.pkl")

2018


E:\Anaconda\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.69339 | val_0_auc: 0.5389  |  0:00:34s
epoch 1  | loss: 0.68992 | val_0_auc: 0.56007 |  0:01:08s
epoch 2  | loss: 0.68702 | val_0_auc: 0.57157 |  0:01:42s
epoch 3  | loss: 0.68439 | val_0_auc: 0.58092 |  0:02:16s
epoch 4  | loss: 0.68192 | val_0_auc: 0.58843 |  0:02:50s
epoch 5  | loss: 0.67983 | val_0_auc: 0.59319 |  0:03:24s
epoch 6  | loss: 0.67751 | val_0_auc: 0.5999  |  0:03:58s
epoch 7  | loss: 0.67582 | val_0_auc: 0.60352 |  0:04:32s
epoch 8  | loss: 0.67577 | val_0_auc: 0.58879 |  0:05:06s
epoch 9  | loss: 0.67399 | val_0_auc: 0.6069  |  0:05:40s
epoch 10 | loss: 0.67214 | val_0_auc: 0.61027 |  0:06:15s
epoch 11 | loss: 0.67071 | val_0_auc: 0.60868 |  0:06:49s
epoch 12 | loss: 0.67006 | val_0_auc: 0.61155 |  0:07:23s
epoch 13 | loss: 0.66903 | val_0_auc: 0.61227 |  0:07:57s
epoch 14 | loss: 0.66902 | val_0_auc: 0.59271 |  0:08:31s
epoch 15 | loss: 0.67005 | val_0_auc: 0.61259 |  0:09:05s
epoch 16 | loss: 0.66721 | val_0_auc: 0.61322 |  0:09:39s
epoch 17 | los

E:\Anaconda\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at ./log/tabnet/2018.zip
2019


...

E:\Anaconda\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at ./log/tabnet/2019.zip
2020


E:\Anaconda\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.69289 | val_0_auc: 0.53463 |  0:01:36s
epoch 1  | loss: 0.69118 | val_0_auc: 0.53456 |  0:03:11s
epoch 2  | loss: 0.69109 | val_0_auc: 0.53943 |  0:04:47s
epoch 3  | loss: 0.68993 | val_0_auc: 0.54863 |  0:06:22s
epoch 4  | loss: 0.68895 | val_0_auc: 0.55781 |  0:07:57s
epoch 5  | loss: 0.68738 | val_0_auc: 0.56573 |  0:09:31s
epoch 6  | loss: 0.68631 | val_0_auc: 0.57121 |  0:11:07s
epoch 7  | loss: 0.68567 | val_0_auc: 0.57076 |  0:12:44s
epoch 8  | loss: 0.68427 | val_0_auc: 0.57764 |  0:14:20s
epoch 9  | loss: 0.68304 | val_0_auc: 0.57803 |  0:15:57s
epoch 10 | loss: 0.68254 | val_0_auc: 0.58056 |  0:17:31s


KeyboardInterrupt: 

In [128]:
from pytorch_tabnet.tab_model import TabNetClassifier

model = TabNetClassifier(device_name='cuda')
model.load_model("./log/tabnet/basic/2018.zip")
model.input_dim

E:\Anaconda\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


982

In [165]:
industry_info = pd.read_pickle("F:\Trade_data\\industry_info.pkl").stack()
industry_list = np.unique(industry_info.values).tolist()
industry_info = industry_info.replace(industry_list, [i for i in range(len(industry_list))])
industry_info = industry_info.loc[factor_stack.index]
industry_info

dt                   
2023-01-03  000001.SZ    34
            000002.SZ    15
            000006.SZ    15
            000007.SZ     8
            000008.SZ    17
                         ..
2023-03-29  688798.SH    22
            688799.SH     7
            688800.SH    22
            688819.SH    21
            688981.SH    22
Length: 252265, dtype: int64

In [185]:
path = "F:\Pfactor_evaluation_results_R5_latest"
file_name_list = os.listdir(path)
ic_mean = []
for file_name in tqdm(file_name_list):
    factor_name = file_name[:-4]
    evaluation = pd.read_pickle(os.path.join(path,file_name))
    ic_series = evaluation['IC_report']['IC_series'].to_frame(factor_name)
    ic_series['year'] = ic_series.index.year
    ic_series = ic_series.groupby('year').mean().rolling(3).mean().loc[2017:]
    ic_series.index = ic_series.index + 1
    ic_series = abs(ic_series)
    ic_mean.append(ic_series)

ic_mean = pd.concat(ic_mean, axis=1)
ic_mean

100%|██████████| 1003/1003 [00:04<00:00, 207.67it/s]


,alphaP0001,alphaP0002,alphaP0003,alphaP0004,alphaP0005,alphaP0006,alphaP0007,alphaP0008,alphaP0009,alphaP0010,...,alphaP2019_5_20,alphaP2019_5_200,alphaP2019_5_5,alphaP2019_5_60,alphaP2019_60_10,alphaP2019_60_120,alphaP2019_60_20,alphaP2019_60_200,alphaP2019_60_5,alphaP2019_60_60
year,,,,,,,,,,,,,,,,,,,,,
2018,0.063553,0.039901,0.018074,0.073046,0.052584,0.035313,0.007961,0.030818,0.006084,0.003972,...,0.026976,0.001015,0.019026,0.007896,0.007249,0.001726,0.020485,0.001474,0.037764,0.006654
2019,0.050471,0.019004,0.011829,0.071386,0.056034,0.036852,0.018776,0.021547,0.008739,0.001909,...,0.024031,0.009792,0.021794,0.011941,0.010966,0.010908,0.020467,0.009506,0.022140,0.010892
2020,0.047403,0.014453,0.012370,0.065773,0.057549,0.036298,0.012923,0.016575,0.011628,0.003454,...,0.025873,0.013163,0.023125,0.012916,0.015647,0.014756,0.022562,0.012882,0.016565,0.011727
2021,0.048106,0.014698,0.018729,0.065230,0.062265,0.029967,0.009449,0.012261,0.009115,0.002975,...,0.026250,0.017083,0.029189,0.019987,0.021343,0.020124,0.023685,0.016778,0.009549,0.018797
2022,0.051454,0.021312,0.026741,0.062490,0.051608,0.025221,0.007967,0.012818,0.006941,0.009698,...,0.035524,0.021602,0.032481,0.025114,0.024454,0.023682,0.031716,0.021226,0.011082,0.023775


In [200]:
selected_factor_dict = {}
for year in ic_mean.index:
    selected_factor = ic_mean.loc[year]
    selected_factor = selected_factor[selected_factor>0.01].index.tolist()
    selected_factor = np.intersect1d(factor_stack.columns, selected_factor)
    selected_factor_dict[year] = selected_factor

pd.to_pickle(selected_factor_dict, "./data/selected_factor_dict.pkl")

In [201]:
selected_factor

array(['alphaP0001', 'alphaP0002', 'alphaP0003', 'alphaP0004',
       'alphaP0005', 'alphaP0006', 'alphaP0008', 'alphaP0012',
       'alphaP0013', 'alphaP1001_10', 'alphaP1001_120', 'alphaP1001_20',
       'alphaP1001_200', 'alphaP1001_5', 'alphaP1001_60', 'alphaP1002_10',
       'alphaP1002_120', 'alphaP1002_20', 'alphaP1002_200',
       'alphaP1002_5', 'alphaP1002_60', 'alphaP1003_120', 'alphaP1004_10',
       'alphaP1004_120', 'alphaP1004_20', 'alphaP1004_200',
       'alphaP1004_5', 'alphaP1004_60', 'alphaP1005_10', 'alphaP1005_120',
       'alphaP1005_20', 'alphaP1005_200', 'alphaP1005_60',
       'alphaP1006_10', 'alphaP1006_120', 'alphaP1006_20',
       'alphaP1006_200', 'alphaP1006_5', 'alphaP1006_60', 'alphaP1007_10',
       'alphaP1007_20', 'alphaP1008_10', 'alphaP1008_120',
       'alphaP1008_20', 'alphaP1008_200', 'alphaP1008_5', 'alphaP1008_60',
       'alphaP1009_10', 'alphaP1009_120', 'alphaP1009_20',
       'alphaP1009_200', 'alphaP1009_5', 'alphaP1009_60', 'alphaP1010_